### Calculating PRS using VCF files


In [101]:
# making the notebok wider
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [10]:
from __future__ import division
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark import SparkConf, SparkContext
# We can give a name to our app (to find it in Spark WebUI) and configure execution mode

APP_NAME="MAVANvcfPRS"
spark=SparkSession.builder.appName(APP_NAME).config("spark.ui.port", "4040").getOrCreate()
#conf = pyspark.SparkConf().setAppName(APP_NAME).set()#.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
sc = spark.sparkContext

sc.setLogLevel("WARN")
log4jLogger = sc._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
LOGGER.info("Start Reading Files")
#def main(gwasFile, genoFileList, thresholdList):

print(spark)
print(sc)

In [58]:
from operator import add
import re
import glob, os
import csv
from collections import Counter
import ntpath
import functools
#from functools import reduce
from math import log
import itertools
import PRS_run as PRS_VCF_utils
from time import time
from copy import deepcopy
#**ATTN: python index starts at 0, so if you want to specify the second column, use 1
#**ATTN: please remove the header of the GWAS file if there is any

# define column number for contents in GWAS

gwas_id=0    # column of SNP ID
gwas_p=7     # column of P value
gwas_or=5    # column of odds ratio
gwas_a1=3    # column of a1 in the GWAS
gwas_maf=10 # column index of maf in the GWAS

# defin column number for contents in genfile
geno_id=2  # column number with rsID
geno_start=9 # column number of the 1st genotype, in the raw vcf files, after separated by the delimiter of choice
geno_a1 = 3  # column number that contains the reference allele


# List of thresholds:
thresholds=[0.5, 0.3, 0.2, 0.1, 0.05, 0.01, 0.001]

# file delimiters:
GWAS_delim="\t"
GENO_delim="\t"

# file names:
home="/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/"  #define homefolder path

# Name of GWAS file 
gwasFiles="/Users/meaneylab/Documents/Nelson/GWAS/pgc.mdd.2012-04/pgc.mdd.clump.withAF.txt"
GWAS_has_header=True

def getFileFromPattern(*pattern): # Multiple patterns need to be put into list format
    files=[]
    for pathpattern in pattern:
        files=glob.glob(files)

genoFileNamePattern=home+"KIDS*info03.vcf"   

genoFileNames=glob.glob(genoFileNamePattern)
# Alternatively, directly specify filename:
#genoFileName=[home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGA.gen",
              #home+"fcgene_out_chr21comb.bierut1M_plus_filtered_chr21_c1_EA_COGEND.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGA.gen",
              #home+"fcgene_out_chr22comb.bierut1M_plus_filtered_chr22_c1_EA_COGEND.gen"]

genoExtension=".vcf"


# programme parameters
log_or=True  # sepcify whether you want to log your odds ratios
check_ref=True # if you know that there are mismatch between the top strand in the genotypes and that of the GWAS, set True. Not checking the reference allele will improve the speed

# sample file path and name
sampleFilePath=home+"MAVAN_35_impute161121_KIDS_orderedSamples.csv" # include the full/relative path and name of the sample file
sampleFileDelim=","  # sample File Delimiter
sampleFileID=0   # which column in the sample file has the ID
sample_skip=1  # how many lines to skip so that the sample names can be matched to the genotypes 1-to-1, taking into account the header of the sample file 
##output file information

outputPath=home+"MAVAN_KIDS_EWBPRS_161129.csv"



In [3]:
print("="*40)
print("Using these genoytpe files: ")

counter = 0
for filename in genoFileNames:
    if counter<20:
        counter+=1
        print(filename)
    else:
        print("and more...")
        print("total of {} files".format(str(len(genoFileNames))))
        break

Using these genoytpe files: 
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS9_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS8_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS7_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS6_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS5_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS4_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS3_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS2_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS22_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS21_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/KIDS_info03/KIDS20_info03.vcf
/Volumes/mavan/Genotyping_161114/MAVAN_imputed_161121/K

### 1. Load files 

In [156]:
genodata=sc.textFile(genoFileNamePattern)
#gwasfile=sc.textFile(gwasFiles)
print("Using the GWAS file: {}".format(ntpath.basename(gwasFiles)))
#gwastable=gwasfile.filter(lambda line: "snpid" not in line).map(lambda line: line.split(GWAS_delim))
gwastableCA=spark.read.option("header", GWAS_has_header).option("delimiter",GWAS_delim).csv(gwasFiles).cache()
gwastable.show(5)



Using the GWAS file: pgc.mdd.clump.withAF.txt
+----------+-------+-------+---+---+------+------+------+-----+---+---------+
|     snpid|hg18chr|     bp| a1| a2|    or|    se|  pval| info|ngt|    CEUaf|
+----------+-------+-------+---+---+------+------+------+-----+---+---------+
|rs12562034|      1| 758311|  A|  G|0.9086| 0.089|0.2817| 1.02|  2|0.0925926|
| rs4475691|      1| 836671|  T|  C| 1.004|0.0534|0.9342|0.981|  2| 0.146789|
| rs2341362|      1| 917172|  T|  C|0.9136|0.0925|0.3284|0.975|  0|0.0275229|
| rs9442372|      1|1008567|  A|  G| 1.025|0.0255|  0.33|0.953|  5| 0.408257|
| rs6671356|      1|1029889|  T|  C| 1.029|0.0359|0.4301|0.978|  0| 0.889908|
+----------+-------+-------+---+---+------+------+------+-----+---+---------+
only showing top 5 rows



### 1.1 Filter GWAS and prepare odds ratio


In [16]:
reload(PRS_VCF_utils)
maxThreshold=max(thresholds)
gwasOddsMapMax=PRS_VCF_utils.filterGWASByP_DF(GWASdf=gwastableCA, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=maxThreshold, logOdds=log_or)
gwasOddsMapMaxCA=sc.broadcast(gwasOddsMapMax).value

Taking the log of odds ratios


### 2. Initial processing 

In [17]:
# at this step, the genotypes are already filtered to keep only the ones in 'gwasOddsMap'
genointermediate=genodata.filter(lambda line: ("#" not in line))\
.map(lambda line: line.split(GENO_delim))\
.filter(lambda line: line[geno_id] in gwasOddsMapMaxCA)\
.map(lambda line: line[0:5]+[chunk.split(":")[3] for chunk in line[geno_start::]])\
.map(lambda line: line[0:5]+[triplet.split(",") for triplet in line[5::]])

genoAlleles=genointermediate.map(lambda line: (line[geno_id], (line[geno_a1], line[geno_a1+1])))
genotable=genointermediate.map(lambda line: (line[geno_id], list(itertools.chain.from_iterable(line[5::]))))\
.mapValues(lambda geno: [float(x) for x in geno])


### 2.1 Calculate and store MAF

In [18]:
reload(PRS_VCF_utils)
genoa1f=genointermediate.map(lambda line: (line[geno_id], (line[geno_a1], line[geno_a1+1]), [float(x) for x in list(itertools.chain.from_iterable(line[5::]))]))\
.map(lambda line: (line[0], line[1][0], line[1][1], PRS_VCF_utils.getMaf(line[2]))).toDF(["Snpid_geno", "GenoA1", "GenoA2", "GenoA1f"])

#genoa1f.map(lambda line:"\t".join([line[0], "\t".join(line[1]), str(line[2])])).saveAsTextFile("../MOMS_info03_maf")


### 3. Determine whether each SNP needs to be flipped

In [29]:
# may take a long time
gwasA1f=gwastableCA.rdd.map(lambda line:(line[gwas_id], line[gwas_a1], line[gwas_a1+1], line[gwas_maf])).toDF(["Snpid_gwas", "GwasA1", "GwasA2", "GwasMaf"])
checktable=genoa1f.join(gwasA1f, genoa1f["Snpid_geno"]==gwasA1f["Snpid_gwas"], "inner").cache()
bpMap={"A":"T", "T":"A", "C":"G", "G":"C"}
reload(PRS_VCF_utils)
flagMap=checktable.rdd.map(lambda line: PRS_VCF_utils.checkAlignmentDF(line, bpMap)).collectAsMap()


### 4. Convert genotype dosages to single-digit representation

In [137]:
samplesize=int(len(genotable.first()[1])/3)
print("Detected {} samples" .format(str(samplesize)))

Detected 261 samples


In [162]:
# filter the raw genotype file
reload(PRS_VCF_utils)
if check_ref:
    print("Generate genotype dosage while taking into account strand alignment differences")
    genotypeMax=genotable.map(lambda line: PRS_VCF_utils.makeGenotypeCheckRef(line, checkMap=flagMap)).cache()
    
else:
    print("Generate genotype dosage without checking strand alignments")
    genotypeMax=genotable.map(lambda line: PRS_VCF_utils.makeGenotype(line, gwasOddsMapCA)).cache()


Generate genotype dosage while taking into account strand alignment differences


### 5. Calculate PRS at the sepcified thresholds

In [164]:
# Calculate the PRS with the maximum threshold
# calculate PRS from genotype
reload(PRS_VCF_utils)
def calcPRSFromGeno(genotypeRDD, oddsMap):
    totalcount=genotypeRDD.count()
    multiplied=genotypeRDD.map(lambda line:[call * oddsMap[line[0]] for call in line[1]])
    PRS=multiplied.reduce(lambda a,b: map(add, a, b))
    normalizedPRS=[x/totalcount for x in PRS]
    return (totalcount,PRS)


In [167]:
def calcAll(genotypeRDD, gwasRDD, thresholdlist):
    prsMap={}
    if len(thresholdlist)>1:
        thresholdNoMaxSorted=sorted(thresholdlist, reverse=True)
    else:
        thresholdNoMaxSorted=thresholdlist
    start=time()
    
    for threshold in thresholdNoMaxSorted:
        tic=time()
        gwasFilteredBC=sc.broadcast(PRS_VCF_utils.filterGWASByP_DF(GWASdf=gwasRDD, pcolumn=gwas_p, idcolumn=gwas_id, oddscolumn=gwas_or, pHigh=threshold, logOdds=log_or))
        #gwasFiltered=spark.sql("SELECT snpid, gwas_or_float FROM gwastable WHERE gwas_p_float < {:f}".format(threshold) 
        print("Filtered GWAS at threshold of {}. Time spent : {:3.1f} seconds".format( str(threshold), time()-tic) )
        checkpoint=time()
        filteredgenotype=genotypeRDD.filter(lambda line: line[0] in gwasFilteredBC.value)
        if not filteredgenotype.isEmpty():
            prsOther=calcPRSFromGeno(filteredgenotype, gwasFilteredBC.value)
            prsMap[threshold]=prsOther
            print("Finished calculating PRS at threshold of {}. Time spent : {:3.1f} seconds".format(str(threshold), time()-checkpoint))
            
    return prsMap

prsDict=calcAll(genotypeMax,gwastableCA, thresholds)

Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.5. Time spent : 1.0 seconds
Finished calculating PRS at threshold of 0.5. Time spent : 3383.0 seconds
Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.3. Time spent : 0.9 seconds
Finished calculating PRS at threshold of 0.3. Time spent : 28.5 seconds
Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.2. Time spent : 0.8 seconds
Finished calculating PRS at threshold of 0.2. Time spent : 24.9 seconds
Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.1. Time spent : 0.8 seconds
Finished calculating PRS at threshold of 0.1. Time spent : 37.9 seconds
Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.05. Time spent : 1.3 seconds
Finished calculating PRS at threshold of 0.05. Time spent : 28.9 seconds
Filtering GWAS, taking the log of odds ratios
Filtered GWAS at threshold of 0.01. Time spent : 1.3 seconds

In [89]:
reload(PRS_VCF_utils)
subjNames=PRS_VCF_utils.getSampleNames(sampleFilePath,sampleFileDelim,sampleFileID, skip=1)

In [94]:

reload(PRS_VCF_utils)
test=PRS_VCF_utils.writePRS(prsDict,  outputPath, samplenames=subjNames)

Successfully wrote scores to MAVAN_KIDS_EWBPRS_161129.csv
